In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick201904final.xlsx")
train1data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick201904train1final.xlsx")
train2data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick201904train2final.xlsx")
train3data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick201904train3final.xlsx")

In [3]:
data1 = alldata.sort_values("LocationID")
data1 = data1.reset_index(drop=True)

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density','X','Y']])
data1[['pop_density','commercial_density','road_density','X','Y']] = scaler.transform(data1[['pop_density','commercial_density','road_density','X','Y']])

In [5]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data1[["time"]])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GTWR
#20, 0.2
results1 = GTWR(coords, t, y, x,95,0.01, kernel='bisquare').fit()

results1.R2

0.9024761944427667

In [6]:
coords = results1.coords
coords = pd.DataFrame(coords)

time = results1.t
time = pd.DataFrame(time)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([time,betas,tvalues,coords], axis = 1)
final_result.columns=['time','beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

In [7]:
time_data = final_result.sort_values("time")

In [8]:
data2 = pd.merge(data1, time_data, left_on=['time','X','Y'], right_on=['time','X','Y'] , how='left')
data2 = data2.sort_values("time")
data2 = data2.reset_index(drop=True)
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)

In [9]:
data3 = pd.DataFrame()
for j in range(len(data2)):
    if abs(data2.iloc[j,8]) > 0 and abs(data2.iloc[j,9]) > 0 and abs(data2.iloc[j,10]) > 0 and abs(data2.iloc[j,11]) > 0 :
        a = data2.iloc[j,:]
        data3 = pd.concat([data3, a], axis=1, ignore_index = True)

data3 = data3.T
beta = data3.groupby('time').mean()
beta.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick201904beta.csv',index=True,sep=',')

In [10]:
data = data3.round(6)

In [11]:
beta1 = data.groupby('LocationID')['POP_beta'].mean()
beta1.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick201904beta1.csv',index=True,header = 0,sep=',')

In [12]:
beta2 = data.groupby('LocationID')['COM_beta'].mean()
beta2.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick201904beta2.csv',index=True,header = 0,sep=',')

In [13]:
beta3 = data.groupby('LocationID')['ROA_beta'].mean()
beta3.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick201904beta3.csv',index=True,header = 0,sep=',')

In [14]:
tvalues

,0,1,2,3
0,0.011398,0.003042,0.003807,-0.008522
1,0.001104,-0.008299,-0.004843,0.004787
2,-0.004697,-0.008908,-0.008340,0.007623
3,0.000509,-0.001189,-0.000536,-0.001032
4,0.000441,-0.001483,-0.000419,-0.001603
...,...,...,...,...
38708,15.714472,4.779201,-2.196707,1.476626
38709,16.124128,3.594318,-2.901644,1.671427
38710,14.795919,2.447223,-2.173462,3.746252
38711,8.394913,3.378580,-1.756046,0.140370
